# Introduction to _netrics_: a Python 2.7 module for the econometric analysis of network data
<br>
_Bryan Graham - University of California - Berkeley_
<br>
_September 2016_ 
<br>
<br>
**Code citation:**
<br>
<br>
Graham, Bryan S. (2016). "Introduction to _netrics_: a Python 2.7 module for the econometric analysis of network data," (Version 1.0) [Computer program]. Available at http://bryangraham.github.io/econometrics/ (Accessed 04 Sept 2016) 
<br>
<br>
**Paper citation:**
<br>
<br>
Graham, Bryan S. (2014). “An econometric model of link formation with degree heterogeneity,” NBER Working Paper No. 20341. Available at http://www.nber.org/papers/w20341 
<br>
<br>
This notebook provides a quick introduction to the main features of the **netrics** package. This package is registered on PyPi, with a GitHub repository at https://github.com/bryangraham/netrics. This package provides an implementation of the _tetrad logit_ and _joint logit fixed effects_ estimators introduced in Graham (2014).
<br>
<br>
The **netrics** package has the following dependencies: numpy, scipy, itertools, pandas, numba and numexpr. These are standard libraries and are included in most scientific Python distributions. For example they are included in the highly recommended Anaconda distribution of Python. If you are using the Anaconda distribution of Python, then you can follow the (straightforward but tedious) instructions here to learn how install the netrics package from PyPi and make it available in Anaconda using the "conda" package manager. For users who anticipate only infrequent use, permanent installation of the **netrics** package may not be worth the trouble. One possibility is to just clone (ie., copy) the GitHub repository https://github.com/bryangraham/netrics, which contains the latest version of **netrics**. Then append the path pointing to the location of the package on your local machine to your sys directory. This is what is done in the next snippet of code.

In [2]:
# Import numpy in order to correctly read test data
import numpy as np

# Import urllib in order to download test data from Github repo
import urllib

# Append location of netrics module base directory to system path
# NOTE: only required if permanent install not made (see comments above)
import sys
sys.path.append('/Users/bgraham/Dropbox/Sites/software/netrics/')

# Load netrics module
import netrics as netrics

For illustration purposes I make use of a cleaned version of the Nyakatoke risk-sharing network data collected by Joachim de Weerdt. The raw files are available for download at https://www.uantwerpen.be/en/staff/joachim-deweerdt/ (where links to research based on the data may also be found). I have placed a cleaned numpy pickle file with an estimation sample on the GitHub repository for the **netrics** package. This next snippet of code downloads this file and prepares the data for input into the *tetrad\_logit()* and *dyad\_jfe\_logit()* functions.

In [3]:
# Download Nyakatoke test dataset from GitHub
download =  '/Users/bgraham/Dropbox/'                               # Edit to download location on your local machine   
url = 'https://github.com/bryangraham/netrics/blob/master/Notebooks/Nyakatoke_Example.npz?raw=true'
urllib.urlretrieve(url, download + "Nyakatoke_Example.npz")

# Open dataset
NyakatokeTestDataset = np.load(download + "Nyakatoke_Example.npz")

# Extract adjacency matrix
D = NyakatokeTestDataset['D']

# Initialize list of dyad-specific covariates as elements
# W = [W0, W1, W2,...WK-1]
W = []

# Initialize list with covariate labels
cov_names = []

# Construct list of regressor matrices and corresponding variable names
for matrix in NyakatokeTestDataset.files:
    if matrix != 'D':
        W.append(NyakatokeTestDataset[matrix])
        cov_names.append(matrix)   

IOError: Failed to interpret file '/Users/bgraham/Dropbox/Nyakatoke_Example.npz' as a pickle

Below I fit the a dyadic link formation model to the Nyakatoke dataset using the two estimators introduced by Graham (2014, _NBER_). The link formation model studied in that paper is


<div> $$\Pr\left(\left.D_{ij}=1\right|\mathbf{X},\mathbf{A}\right) = \frac{\exp\left(\sum_{k=1}^{K}W_{k,ij}\beta_{k}+A_{i}+A_{j}\right)}   {1+\exp\left(\sum_{k=1}^{K}W_{k,ij}\beta_{k}+A_{i}+A_{j}\right)}$$ </div>

Here $\mathbf{X}$ denotes all the household-specific observed covariates in the network, $W_{k,ij} = W_{k,ji}$ are dyad-specific covariates constructed from $\mathbf{X}$, and the $A_i$ for $i=1,...,N$ are _degree heterogeneity_ parameters which are household specific. These are treated as co-called "fixed effects".
<br>
<br>
Graham (2014) introduces a "tetrad logit" procedure whose criterion function is invariant to the fixed effects and also a "joint logit fixed effects" procedure which estimates the individual effects along with the common parameters.
<br>
<br>
In the **netrics** package these two procedures are operationalized by the *tetrad\_logit()* and *dyad\_jfe\_logit()* functions. These two functions require the user to input data in a very particular way. The outcome is $\mathbf{D}$, the $N \times N$ adjacency matrix. The regressors are included in a length $K$ list where each element is an $N \times N $ numpy 2d array $W_{k}$ with $(i,j)$ element equal to the dyadic covariate $W_{k,ij}$. 
<br>
<br>

## Tetrad Logit

A help call gives some basic information about *tetrad\_logit()*.

In [3]:
help(netrics.tetrad_logit)

Help on function tetrad_logit in module netrics.tetrad_logit:

tetrad_logit(D, W, dtcon=None, silent=False, W_names=None)
    AUTHOR: Bryan S. Graham, bgraham@econ.berkeley.edu, June 2016
    
    This function computes the Tetrad Logit estimator introduced in Graham (2014, NBER No. 20341) -- "An Econometric
    Model of Link Formation with Degree Heterogeneity". The implementation is as described in the paper. Notation
    attempts to follow that used in the paper.
    
    INPUTS
    ------
    D                 : N x N undirected adjacency matrix
    W                 : List with elements consisting of N x N 2d numpy arrays of dyad-specific 
                        covariates such that W[k][i,j] gives the k-th covariate for dyad ij
    dtcon             : Dyad and tetrad concordance (dtcon) List with elements [tetrad_to_dyads_indices, 
                        dyad_to_tetrads_dict]. If dtcon=None, then construct it using generate_tetrad_indices() 
                        function. Se

Next call tetrad_logit(). Depending on the speed and memory of your computer, this next bit of code may take a few minutes to complete.

In [4]:
[beta_TL, vcov_beta_TL, tetrad_frac_TL, success] = \
    netrics.tetrad_logit(D, W, dtcon=None, silent=False, W_names=cov_names)

Fisher-Scoring Derivative check (2-norm): 4.71127425
Value of -logL = 138316.376938,  2-norm of score = 10100196.293329
Value of -logL = 135187.248221,  2-norm of score = 1596218.251378
Value of -logL = 135076.189036,  2-norm of score = 455357.680707
Value of -logL = 134067.546456,  2-norm of score = 360772.909168
Value of -logL = 132607.914482,  2-norm of score = 195723.087456
Value of -logL = 132227.469603,  2-norm of score = 90172.252165
Value of -logL = 119215.459180,  2-norm of score = 774170.803762
Value of -logL = 119190.326354,  2-norm of score = 29152.067645
Value of -logL = 115270.688300,  2-norm of score = 100160.420505
Value of -logL = 115270.231605,  2-norm of score = 21801.989550
Value of -logL = 110870.099445,  2-norm of score = 129096.779630
Value of -logL = 110869.345527,  2-norm of score = 16812.212735
Value of -logL = 108924.012960,  2-norm of score = 28476.501423
Value of -logL = 105502.848918,  2-norm of score = 110760.679917
Value of -logL = 105502.272196,  2-norm

The program spits out some useful information. In a network with $N = 115$ agents there are a total of $6,913,340$ different tetrads! Of these only $102,151$, or about 1.5 percent, actually contribute to the *tetrad\_logit()* criterion function. The effectiveness of the procedure in the context of sparse networks is one of its theoretical and practical attractions.
<br>
<br>
The results suggest that ties are more frequent between households which are physically close, related by blood and where the household heads are close in age. There is little evidence for sorting by religion, clan or education. There is some evidence for heterophily in terms of wealth (perhaps consistent with some models of "mutual support"), but it is rather weak.
<br>
<br>
The *tetrad\_logit()* estimation procedure is computationally intense. On a modern desktop machine it would be difficult to use the procedure on a network larger than a few hundred agents (fortunately there are many possible ways to apply the method at scale via various approximations and parallelizations; but these extensions are not (yet) part of the **netrics** package). 
<br>
<br>
Part of the computational intensity is up front. Specifically, the function computes a detailed dictionary which maps dyads-to-tetrads and vice-versa. This bookkeeping overhead makes later calculations much quicker. A user who anticipates fitting several different models to the same adjacency matrix can save considerable time by computing this concordance first (once and for all) and then passing it to *tetrad\_logit()* via the _dtcon_ parameter. This can be done by calling the *generate\_tetrad\_indices()* function.
<br>
<br>
To illustrate I create the concordance and then re-fit the model.

In [5]:
N = np.shape(D)[0]
concordance = netrics.generate_tetrad_indices(N, full_set=True)
[beta_TL, vcov_beta_TL, tetrad_frac_TL, success] = \
    netrics.tetrad_logit(D, W, dtcon=concordance, silent=False, W_names=cov_names)

Fisher-Scoring Derivative check (2-norm): 4.71127425
Value of -logL = 138316.376938,  2-norm of score = 10100196.293329
Value of -logL = 135187.248221,  2-norm of score = 1596218.251378
Value of -logL = 135076.189036,  2-norm of score = 455357.680707
Value of -logL = 134067.546456,  2-norm of score = 360772.909168
Value of -logL = 132607.914482,  2-norm of score = 195723.087456
Value of -logL = 132227.469603,  2-norm of score = 90172.252165
Value of -logL = 119215.459180,  2-norm of score = 774170.803762
Value of -logL = 119190.326354,  2-norm of score = 29152.067645
Value of -logL = 115270.688300,  2-norm of score = 100160.420505
Value of -logL = 115270.231605,  2-norm of score = 21801.989550
Value of -logL = 110870.099445,  2-norm of score = 129096.779630
Value of -logL = 110869.345527,  2-norm of score = 16812.212735
Value of -logL = 108924.012960,  2-norm of score = 28476.501423
Value of -logL = 105502.848918,  2-norm of score = 110760.679917
Value of -logL = 105502.272196,  2-norm

## Joint Fixed Effects Logit

Graham (2014) also introduces a joint fixed effects estimator for link formation. This procedure estimates the coefficients on $W_{k,ij}$ for $k=1,...,K$ _as well as_ individual specific degree heterogeneity parameters $A_{i}$ for $i=1,...,N$. The *dyad\_jfe\_logit()* function implements this estimator. By default if reports the iterated bias-corrected estimates described in the paper (but the uncorrected estimates are also returned by the function). I use the tetrad logit point estimates as starting values.

In [6]:
[beta_JFE, beta_JFE_BC, vcov_beta_JFE, A_JFE, success] = \
    netrics.dyad_jfe_logit(D, W, T=None, silent=False, W_names=cov_names, beta_sv=beta_TL)

-------------------------------------------------------------------------------------------
- COMPUTE JOINT FIXED EFFECT MLEs                                                         -
-------------------------------------------------------------------------------------------
Value of c_logl = 1366.571003,  2-norm of c_score = 6874.838541
Value of c_logl = 1366.036417,  2-norm of c_score = 5098.916794
Value of c_logl = 1365.735729,  2-norm of c_score = 3780.317947
Value of c_logl = 1365.567620,  2-norm of c_score = 2801.876363
Value of c_logl = 1365.474058,  2-norm of c_score = 2076.136397
Value of c_logl = 1365.422158,  2-norm of c_score = 1537.983914
Value of c_logl = 1365.393435,  2-norm of c_score = 1139.016582
Value of c_logl = 1365.377561,  2-norm of c_score = 843.292195
Value of c_logl = 1365.368791,  2-norm of c_score = 624.140106
Value of c_logl = 1365.363939,  2-norm of c_score = 461.781190
Value of c_logl = 1365.361246,  2-norm of c_score = 341.555992
Value of c_logl = 1365.3

It takes some time for *dyad\_jfe\_logit()* to converge (even with the carefully chosen starting values). This reflects the sparsity of the Nyakatoke network and the consequent difficulty of estimating the household degree heterogeneity effects.
<br>
<br>
Relative to the *tetrad\_logit()* results, the joint estimator suggests sorting by religion and clan. In Monte Carlo experiments I have found that the joint estimator -- particularly the bias correction step -- can work very poorly in networks like the Nyakatoke one (i.e., in networks that are sparse such that many agents have only a few links). In such settings the $A_{i}$ for $i=1,...,N$ may be _very_ imprecisely estimated and this can effect the quality of the common parameter estimates as well.
<br>
<br>
For reference we can look at the joint coefficient estimates prior to bias correction using the *print\_coef()* utility included in the **netrics** package. These point estimates are closer to the *tetrad\_logit()* ones than their bias-corrected counterparts. This suggests, that in this particular example, bias-correction may be doing more harm that good.
<br>
<br>
Monte Carlo evidence suggests that in denser networks, the size distortion causes by bias in the limit distribution of the joint fixed effects estimator is very real and, furthermore, that bias correction can be effective in such contexts. So not too much should be made of this example.

In [1]:
netrics.print_coef(beta_JFE, vcov_beta_JFE, var_names=cov_names)

NameError: name 'netrics' is not defined

The goal is to add additional functionality to the package over time. In the meantime please feel free to touch base with bug reports and/or suggestions for improvements. Unfortunately I cannot provide any meaningful user support.

In [8]:
# This imports an attractive notebook style from Github
from IPython.core.display import HTML
import urllib2
HTML(urllib2.urlopen('http://bit.ly/1Bf5Hft').read())